In [2]:
import os
import openai
from openai import OpenAI

In [3]:
from dotenv import load_dotenv 
load_dotenv()

True

In [4]:
# 환경 변수에서 API 키 가져오기
openai_api_key = os.getenv("emogi_openai_api_key")

In [5]:
from pathlib import Path
from langchain_openai import ChatOpenAI 
# from langchain_upstage import ChatUpstage
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain.memory import ConversationBufferMemory
from operator import itemgetter

In [6]:
import pkg_resources
installed_packages = pkg_resources.working_set
langchain_version = [(d.project_name, d.version) for d in installed_packages if 'langchain' in d.project_name]
print(langchain_version)

[('langchain', '0.2.7'), ('langchain-community', '0.2.7'), ('langchain-core', '0.2.12'), ('langchain-google-genai', '1.0.7'), ('langchain-openai', '0.1.14'), ('langchain-teddynote', '0.0.35'), ('langchain-text-splitters', '0.2.2')]


In [7]:
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("emogi_openai")

LangSmith 추적을 시작합니다.
[프로젝트명]
emogi_openai


In [8]:
def read_luckyvicky(x):
    luckyvicky_path = Path("../data/persona/luckyvicky.txt")
    return luckyvicky_path.read_text(encoding="utf-8")

template = """\
# INSTRUCTION
- 당신의 이름은 ooo입니다. 
- 당신의 성격은 PERSONALITY와 같습니다.
- PERSONALITY에 맞춰 USER에 답변하세요.

# PERSONALITY: {personality}

# USER: {input}
"""

from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chat_models import ChatOpenAI
from langchain.schema import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
from operator import itemgetter

In [9]:
# 환경 변수에서 API 키 읽기
openai_api_key = os.getenv("emogi_openai_api_key")
if openai_api_key is None:
    raise ValueError("emogi_openai_api_key 환경 변수가 설정되지 않았습니다.")

memory = ConversationBufferMemory(return_messages=True, memory_key="chat_history")
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", template),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),
    ]
)

In [10]:
# ChatOpenAI 모델 초기화 (gpt-4o-mini 사용)
model = ChatOpenAI(model_name="gpt-4o-mini", temperature=0.7, api_key=openai_api_key)

output_parser = StrOutputParser()
runnable1 = {"input": RunnablePassthrough()}
runnable2 = RunnablePassthrough.assign(
    chat_history=RunnableLambda(memory.load_memory_variables) | itemgetter("chat_history"),
    personality=RunnableLambda(read_luckyvicky)
)
runnable = runnable1 | runnable2
chain = runnable | prompt | model | StrOutputParser()

d:\envs\want2\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [ ]:
# 대화 함수
def chat_with_model(input_text):
    result = chain.invoke({"input": input_text})
    # 대화 기록 저장
    memory.chat_memory.add_user_message(input_text)
    memory.chat_memory.add_ai_message(result)
    return result

# 사용자 입력 받아서 대화하기
print("대화를 시작합니다. 종료하려면 'exit', 'quit', 또는 '종료'를 입력하세요.")
while True:
    user_input = input("사용자: ")
    if user_input.lower() in ["exit", "quit", "종료"]:
        print("대화를 종료합니다.")
        break
    response = chat_with_model(user_input)
    print(f"원영적 사고: {response}\n")

대화를 시작합니다. 종료하려면 'exit', 'quit', 또는 '종료'를 입력하세요.
원영적 사고: 헬로우~ 반가워! 😄✨ 오늘도 수다 떨 준비됐지? 날씨가 추운 건 조금 힘들 수 있지만, 오히려 좋아! 이렇게 추운 날은 따뜻한 음료를 즐길 기회잖아! ☕️😊 너는 따뜻하게 잘 지내고 있어? 어떤 음료 좋아해? ❤️

